---
title: "30 Day Map Challenge 2024 - Day 14: A world map"
categories:
  - Project
tags:
  - 30daymapchallenge
  - python
  - programming
classes: wide
header:
  teaser: /assets/images/30daymapchallenge2024-day14.png
---

The theme for day 14 is _A world map_:
> Map the whole world. Whether it’s continents, ecosystems, or oceans, this is the day to map the entire planet.

### Data

Today, we will use data from the [_PANGAEA_ data warehouse](https://www.pangaea.de/).
I found that the search enginge of the EUDAT's b2find service is superior to that of PANGAEA.
Therefore, I will query for "Polarstern Master Track" there, scrape the results and use the links to the PANGAEA website to locate the actual dataset.
Using a python library, we can then download the datasets.


### Data Download

```shell
curl 'https://search-es-wmm-prod-4wpjacc7xjk4fixaluzofej5vq.eu-west-1.es.amazonaws.com/rankings/_search' --data-raw '{"from":"0","size":"1000000","sort":[{"_score":{"order":"desc"}},{"fastest_finish_time_secs":{"order":"asc"}}],"query":{"bool":{"must":[],"filter":[{"match":{"edition":"6"}},{"match":{"gender":"M"}}],"must_not":{"match":{"overall_ranking":0}}}}}' -H 'content-type: application/json' > results.json
```

## Implementation

Create a dataframe that holds the average income per inhabitant of the city Dresden (Germany). Each row contains the data for one district of Dresden.

In [ ]:
import json
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from geopy.geocoders import Nominatim
import pycountry
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

In [ ]:
data_folder = Path(f"data/abbott/2024worldranking")

In [ ]:
for sex in ("m", "f"):
    data_file_path = data_folder / f"results_{sex}.json"
    with open(data_file_path) as data_file:
        data = json.load(data_file)
    df = pd.concat([pd.json_normalize(s['_source']).loc[:, ["age_group", "gender", "country_image", "fastest_finish_time_secs"]] for s in tqdm(data["hits"]["hits"])])
    df.to_csv(data_file_path.with_suffix(".csv"), index=False)

In [ ]:
df = pd.concat(
    [pd.read_csv(data_folder / f"results_{sex}.csv") for sex in ("m", "f")],
    axis=0
).reset_index(drop=True).query("~country_image.isna()")

In [ ]:
mean_all = df[["country_image", "fastest_finish_time_secs"]].groupby("country_image").mean()

In [ ]:
<Record: <MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, -52.556 2.505, -52.94 2.125, ...>, {'featurecla': 'Admin-0 country', 'scalerank': 1, 'LABELRANK': 2, 'SOVEREIGNT': 'France', 'SOV_A3': 'FR1', 'ADM0_DIF': 1, 'LEVEL': 2, 'TYPE': 'Country', 'TLC': '1', 'ADMIN': 'France', 'ADM0_A3': 'FRA', 'GEOU_DIF': 0, 'GEOUNIT': 'France', 'GU_A3': 'FRA', 'SU_DIF': 0, 'SUBUNIT': 'France', 'SU_A3': 'FRA', 'BRK_DIFF': 0, 'NAME': 'France', 'NAME_LONG': 'France', 'BRK_A3': 'FRA', 'BRK_NAME': 'France', 'BRK_GROUP': '', 'ABBREV': 'Fr.', 'POSTAL': 'F', 'FORMAL_EN': 'French Republic', 'FORMAL_FR': '', 'NAME_CIAWF': 'France', 'NOTE_ADM0': '', 'NOTE_BRK': '', 'NAME_SORT': 'France', 'NAME_ALT': '', 'MAPCOLOR7': 7, 'MAPCOLOR8': 5, 'MAPCOLOR9': 9, 'MAPCOLOR13': 11, 'POP_EST': 67059887.0, 'POP_RANK': 16, 'POP_YEAR': 2019, 'GDP_MD': 2715518, 'GDP_YEAR': 2019, 'ECONOMY': '1. Developed region: G7', 'INCOME_GRP': '1. High income: OECD', 'FIPS_10': 'FR', 'ISO_A2': '-99', 'ISO_A2_EH': 'FR', 'ISO_A3': '-99', 'ISO_A3_EH': 'FRA', 'ISO_N3': '-99', 'ISO_N3_EH': '250', 'UN_A3': '250', 'WB_A2': 'FR', 'WB_A3': 'FRA', 'WOE_ID': -90, 'WOE_ID_EH': 23424819, 'WOE_NOTE': 'Includes only Metropolitan France (including Corsica)', 'ADM0_ISO': 'FRA', 'ADM0_DIFF': '', 'ADM0_TLC': 'FRA', 'ADM0_A3_US': 'FRA', 'ADM0_A3_FR': 'FRA', 'ADM0_A3_RU': 'FRA', 'ADM0_A3_ES': 'FRA', 'ADM0_A3_CN': 'FRA', 'ADM0_A3_TW': 'FRA', 'ADM0_A3_IN': 'FRA', 'ADM0_A3_NP': 'FRA', 'ADM0_A3_PK': 'FRA', 'ADM0_A3_DE': 'FRA', 'ADM0_A3_GB': 'FRA', 'ADM0_A3_BR': 'FRA', 'ADM0_A3_IL': 'FRA', 'ADM0_A3_PS': 'FRA', 'ADM0_A3_SA': 'FRA', 'ADM0_A3_EG': 'FRA', 'ADM0_A3_MA': 'FRA', 'ADM0_A3_PT': 'FRA', 'ADM0_A3_AR': 'FRA', 'ADM0_A3_JP': 'FRA', 'ADM0_A3_KO': 'FRA', 'ADM0_A3_VN': 'FRA', 'ADM0_A3_TR': 'FRA', 'ADM0_A3_ID': 'FRA', 'ADM0_A3_PL': 'FRA', 'ADM0_A3_GR': 'FRA', 'ADM0_A3_IT': 'FRA', 'ADM0_A3_NL': 'FRA', 'ADM0_A3_SE': 'FRA', 'ADM0_A3_BD': 'FRA', 'ADM0_A3_UA': 'FRA', 'ADM0_A3_UN': -99, 'ADM0_A3_WB': -99, 'CONTINENT': 'Europe', 'REGION_UN': 'Europe', 'SUBREGION': 'Western Europe', 'REGION_WB': 'Europe & Central Asia', 'NAME_LEN': 6, 'LONG_LEN': 6, 'ABBREV_LEN': 3, 'TINY': -99, 'HOMEPART': 1, 'MIN_ZOOM': 0.0, 'MIN_LABEL': 1.7, 'MAX_LABEL': 6.7, 'LABEL_X': 2.552275, 'LABEL_Y': 46.696113, 'NE_ID': 1159320637, 'WIKIDATAID': 'Q142', 'NAME_AR': 'فرنسا', 'NAME_BN': 'ফ্রান্স', 'NAME_DE': 'Frankreich', 'NAME_EN': 'France', 'NAME_ES': 'Francia', 'NAME_FA': 'فرانسه', 'NAME_FR': 'France', 'NAME_EL': 'Γαλλία', 'NAME_HE': 'צרפת', 'NAME_HI': 'फ़्रान्स', 'NAME_HU': 'Franciaország', 'NAME_ID': 'Prancis', 'NAME_IT': 'Francia', 'NAME_JA': 'フランス', 'NAME_KO': '프랑스', 'NAME_NL': 'Frankrijk', 'NAME_PL': 'Francja', 'NAME_PT': 'França', 'NAME_RU': 'Франция', 'NAME_SV': 'Frankrike', 'NAME_TR': 'Fransa', 'NAME_UK': 'Франція', 'NAME_UR': 'فرانس', 'NAME_VI': 'Pháp', 'NAME_ZH': '法国', 'NAME_ZHT': '法國', 'FCLASS_ISO': 'Admin-0 country', 'TLC_DIFF': '', 'FCLASS_TLC': 'Admin-0 country', 'FCLASS_US': '', 'FCLASS_FR': '', 'FCLASS_RU': '', 'FCLASS_ES': '', 'FCLASS_CN': '', 'FCLASS_TW': '', 'FCLASS_IN': '', 'FCLASS_NP': '', 'FCLASS_PK': '', 'FCLASS_DE': '', 'FCLASS_GB': '', 'FCLASS_BR': '', 'FCLASS_IL': '', 'FCLASS_PS': '', 'FCLASS_SA': '', 'FCLASS_EG': '', 'FCLASS_MA': '', 'FCLASS_PT': '', 'FCLASS_AR': '', 'FCLASS_JP': '', 'FCLASS_KO': '', 'FCLASS_VN': '', 'FCLASS_TR': '', 'FCLASS_ID': '', 'FCLASS_PL': '', 'FCLASS_GR': '', 'FCLASS_IT': '', 'FCLASS_NL': '', 'FCLASS_SE': '', 'FCLASS_BD': '', 'FCLASS_UA': ''}, <fields>>

In [ ]:
geometries = []
finish_times = []
shpfilename = shpreader.natural_earth(resolution='110m',
                                          category='cultural',
                                          name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()
for country in countries:
    a3 = country.attributes["ADM0_A3"]
    geometries.append(country.geometry)
    if a3 in mean_all.index:
        finish_times.append(mean_all.loc[a3, "fastest_finish_time_secs"])
    else:
        finish_times.append(0.0)
        display(country)

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(projection=ccrs.PlateCarree())

cmap = mcolors.LinearSegmentedColormap.from_list("", ["#effbef","green","darkgreen"])
cmap.set_bad("gray")

art = ax.add_geometries(
    geometries,
    crs=ccrs.PlateCarree(),
    array=finish_times,
    cmap=cmap,
    norm=mcolors.LogNorm(vmin=mean_all["fastest_finish_time_secs"].min(), vmax=mean_all["fastest_finish_time_secs"].max())
)
cbar = fig.colorbar(art, orientation="horizontal", shrink=0.5)
cbar.set_label("Mean fastest finish time (s)")

plt.show()